In [1]:
import os
import sys

# Ensure correct path to project root
sys.path.insert(0, os.path.abspath(".."))

import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'mtihaniapi.settings')
django.setup()

In [2]:
from gen.curriculum import get_cbc_grouped_questions
from gen.utils import *
from mistralai import Mistral
import json
import re


In [3]:
grouped_questions = get_cbc_grouped_questions(
    strand_ids=[2,4,1,6,3,8,9],
    question_count=10,
    bloom_skill_count=2,
    is_debug=True,
)


✅ Question breakdown written to /Users/melaniefayne/Desktop/mtihani/mtihani_api/mtihaniapi/gen/output/question_breakdown.json. Total: 6


In [4]:
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
client = Mistral(api_key=MISTRAL_API_KEY)

MISTRAL_LARGE = "mistral-large-latest"
MISTRAL_MEDIUM = "mistral-medium-latest"
MISTRAL_SMALL = "mistral-small-latest"

In [5]:
def generate_sub_strand_questions(
    sub_strand_data: Dict[str, Any],
    llm: str,
) -> str:
    prompt_template = CREATE_EXAM_LLM_PROMPT
    formatted_prompt = prompt_template.format(
        strand=sub_strand_data["strand"],
        sub_strand=sub_strand_data["sub_strand"],
        learning_outcomes=sub_strand_data["learning_outcomes"],
        skills_to_assess=sub_strand_data["skills_to_assess"],
        skills_to_test=sub_strand_data["skills_to_test"],
        question_count=sub_strand_data["question_count"],
        sample_questions=sub_strand_data["sample_questions"],
    )
    response = client.chat.complete(
        model=llm,
        messages=[{"role": "user", "content": formatted_prompt}],
        temperature=0.1,
        max_tokens=10240,
    )
    
    return response.choices[0].message.content

In [6]:
def safe_parse_llm_output(raw):
    # If it's already a list, return as is
    if isinstance(raw, list):
        return raw
    # If it's a string, clean and parse
    if isinstance(raw, str):
        # Remove Markdown code block formatting if present
        code_block_pattern = r"```(?:json)?\s*([\s\S]*?)```"
        match = re.search(code_block_pattern, raw.strip())
        json_str = match.group(1).strip() if match else raw.strip()
        try:
            return json.loads(json_str)
        except Exception as e:
            print(f"Warning: Could not parse LLM output as JSON. Error: {e}")
            return []
    # For anything else, return empty list
    return []

In [7]:
from asgiref.sync import sync_to_async

@sync_to_async
def get_reference_for_sub_strand(sub_strand: str) -> str:
    from rag.models import SubStrandReference
    try:
        ref = SubStrandReference.objects.get(sub_strand=sub_strand)
        return ref.reference_text or ""
    except SubStrandReference.DoesNotExist:
        return ""

In [ ]:
async def generate_llm_question_list(
    grouped_question_data: List[Dict[str, Any]],
    llm: Any,
    output_file: str = QUESTION_LIST_OUTPUT_FILE,
) -> Union[List[Dict[str, Any]], Dict[str, Any]]:
    all_question_list = []

    for group in grouped_question_data:
        strand = group["strand"]
        sub_strand = group["sub_strand"]
        learning_outcomes = "\n- " + "\n- ".join(group["learning_outcomes"])
        skills_to_assess = "\n- " + "\n- ".join(group["skills_to_assess"])
        sample_questions = await get_reference_for_sub_strand(sub_strand)

        # Step 1: Flatten all skills with their associated breakdown number
        numbered_skills = []
        for entry in group["skills_to_test"]:
            number = entry["number"]
            for skill in entry["skills_to_test"]:
                numbered_skills.append({"number": number, "skill": skill})

        # Step 2: Build a flat list of just the skills (in order)
        skills_only = [entry["skill"] for entry in numbered_skills]

        # Step 3: Generate all questions in one LLM call
        sub_strand_data = {
            "question_count": len(skills_only),
            "strand": strand,
            "sub_strand": sub_strand,
            "learning_outcomes": learning_outcomes,
            "skills_to_assess": skills_to_assess,
            "skills_to_test": skills_only,
            "sample_questions": sample_questions,
        }

        print(f"\n{sub_strand} =========")

        parsed_output = generate_sub_strand_questions(
            sub_strand_data=sub_strand_data,
            llm=llm,
        )

        parsed_output = safe_parse_llm_output(parsed_output)
        if not parsed_output:
            print("Skipping: could not parse output.")
            continue

        if not parsed_output:
            print(
                f"Skipping sub-strand: {sub_strand} due to un-parse-able output")
            continue

        tagged_responses = []
        for idx, qa in enumerate(parsed_output):
            # Prevent index overflow if LLM returns fewer/more questions
            if idx >= len(numbered_skills):
                break
            question_item = {}
            question_item["number"] = numbered_skills[idx]["number"]
            question_item["grade"] = group["grade"]
            question_item["strand"] = group["strand"]
            question_item["sub_strand"] = group["sub_strand"]
            question_item["bloom_skill"] = numbered_skills[idx]["skill"]
            question_item["description"] = qa["question"]
            question_item["expected_answer"] = qa["expected_answer"]
            tagged_responses.append(question_item)
        all_question_list.extend(tagged_responses)

    all_question_list = sorted(all_question_list, key=itemgetter("number"))

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(all_question_list, f, ensure_ascii=False, indent=4)
    print(
        f"\n✅ Question list written to {output_file}. Total: {len(all_question_list)}")

    return all_question_list

In [9]:
selected_model = MISTRAL_SMALL
all_question_list = await generate_llm_question_list(
    grouped_question_data=grouped_questions,
    llm=selected_model,
)

print(all_question_list)

# If there was a generation error
if not isinstance(all_question_list, list):
    print(all_question_list)


Electrical Energy =========
Q: How would you set up a model home to show the generation, use and safety of electrical energy? Explain the sources, appliances and safety measures and discuss how this helps appreciate the role of electricity in daily life.
A: Set up a model home with a small generator or solar panel to demonstrate electricity generation. Use common appliances like lights, fans, and a small heater to show electricity use. Include safety measures like circuit breakers and proper insulation. This setup helps learners understand the importance of electricity in daily life and the need for safety.

Q: Describe the energy transformations that occur from the wind to the powering of a household light bulb in this village.
A: The wind's kinetic energy is transformed into mechanical energy by the windmill. This mechanical energy is then converted into electrical energy by a generator. The electrical energy is stored in a battery and later used to power a household light bulb, con

In [10]:
exam_questions = get_db_question_objects(
    all_question_list=all_question_list,
)

In [11]:
OUTPUT_FILE = os.path.join(
    BASE_DIR, "output/model_comparisons", f"{selected_model}_EXAM.txt")

def export_exam_questions_to_txt(exam_questions, output_file=OUTPUT_FILE):
    with open(output_file, "w", encoding="utf-8") as f:
        for idx, q in enumerate(exam_questions, start=1):
            f.write(f"\nQuestion {idx}\n")
            f.write("-" * 20 + "\n")
            f.write(f"Grade: {q.get('grade', '')}\n")
            f.write(f"Strand: {q.get('strand', '')}\n")
            f.write(f"Sub Strand: {q.get('sub_strand', '')}\n\n")
            
            # Handle single or multiple bloom skills per question
            bloom_skills = q.get("bloom_skills", [])
            questions = q.get("questions", [])
            answers = q.get("expected_answers", [])
            
            # If your structure is per-skill, per-QA:
            for b_idx, (bloom, ques, ans) in enumerate(zip(bloom_skills, questions, answers), start=1):
                f.write(f"Bloom Skill {b_idx}: {bloom}\n")
                f.write(f"      Q: {ques}\n")
                f.write(f"      A: {ans}\n\n")
            
            # If your structure is single-skill per question, you can do:
            # f.write(f"Bloom Skill 1: {q.get('bloom_skill', '')}\n")
            # f.write(f"      Q: {q.get('description', '')}\n")
            # f.write(f"      A: {q.get('expected_answer', '')}\n\n")
                
        print(f"✅ Exam questions exported to {output_file}")

# Usage:
# exam_questions = [...]  # your processed list of dicts
export_exam_questions_to_txt(exam_questions)

✅ Exam questions exported to /Users/melaniefayne/Desktop/mtihani/mtihani_api/mtihaniapi/gen/output/model_comparisons/mistral-small-latest_EXAM.txt
